In [182]:
import cv2
import numpy as np
import math
import random
import keyboard

import time

In [183]:
def range(start,stop=None,step=1):
    if stop==None:
        stop = start
        start = 0 
    i = start
    if step>0:
        while i<stop:
            yield i
            i+=step
    else:
        while i>stop:
            yield i
            i+=step

In [184]:

def place_car(img, X,Y,voiture):
    hight=math.ceil(voiture.shape[0]/2)
    width=math.ceil(voiture.shape[1]/2)
    if (X-width>=0 and Y-hight>=0 and X+width<img.shape[1] and Y+hight<img.shape[0]) :
        img[Y-hight:Y + hight, X-width:X + width, :] = voiture 
    else:
        if((X-width<0 and X+width>img.shape[1]) or (Y-hight<0 and Y+hight>img.shape[0]) ):
            return img
            
    return img

def remove_car(img, X,Y,voiture):
    game_over=False
    hight=math.ceil(voiture.shape[0]/2)
    width=math.ceil(voiture.shape[1]/2)
    if (X-width>=0 and Y-hight>=0 and X+width<img.shape[1] and Y+hight<img.shape[0]) :
        img[Y-hight:Y + hight, X-width:X + width, :] = np.zeros((voiture.shape[0],voiture.shape[1],3)) 
    return img
def place_meteore(img,liste_meteor):
    for i in liste_meteor:
        img=place_car(img,i.posX,i.posY,i.img)
    return img
def remove_meteore(img,liste_meteor):
    for i in liste_meteor:
        img=remove_car(img,i.posX,i.posY,i.img)
    return img




In [185]:
class Meteore:
    def __init__(self, img,posX,posY,moveX,moveY):
        self.img = img
        self.posX=posX
        self.posY=posY
        self.moveX=moveX
        self.moveY=moveY
        self.radius=int(abs(posX-posY)/2)
    def update(self):
        self.posX+=self.moveX
        self.posY+=self.moveY
    def update_posY(self,moveY):
        self.posY=moveY
    def update_posX(self,moveX):
        self.posX=moveX
    def update_moveY(self,moveY):
        self.moveY=moveY
    def update_moveX(self,moveX):
        self.moveX=moveX
    def greet(self):
        print(f"Hello, {self.name}!")
class Voiture:
    def __init__(self, img,posX,posY,speed):
        self.img = img
        self.posX=posX
        self.posY=posY
        self.speed=speed
        self.radius=int(abs(posX-posY)/2)



In [186]:
liste_meteor=[]
for i in range(1):
    met=np.zeros((2*random.randint(10,50),2*random.randint(10,50),3))
    for w in range(met.shape[0]):
        for h in range(met.shape[1]):
            met[w][h][:]=[255,255,255]
    positiony=random.randint(0,300)
    positionx=random.randint(100,540)
    liste_meteor.append(Meteore(met,positionx,positiony,3,5))

list_star=[]
for i in range(1):
    met=np.zeros((2*random.randint(10,50),2*random.randint(10,50),3))
    for w in range(met.shape[0]):
        for h in range(met.shape[1]):
            met[w][h][:]=[255,0,0]
    positiony=random.randint(0,300)
    positionx=random.randint(100,540)
    list_star.append(Meteore(met,positionx,positiony,3,5))

In [187]:
for i in liste_meteor:
    print(i.posX,i.posY,i.moveX,i.moveY,i.radius)
    print(i.img.shape)
    print("")



400 247 3 5 76
(24, 86, 3)



In [188]:
liste_meteor=[]

In [189]:

voitureimg=np.zeros((100,100,3))
for w in range(voitureimg.shape[0]):
    for h in range(voitureimg.shape[1]):
        voitureimg[w][h][:]=[255,255,255]

voiture=Voiture(voitureimg,320,400,10)


scoreborder=np.zeros((200,700,3))
cv2.putText(scoreborder,"score : 0",(10,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)


obstaclespeed=100

In [190]:
start=True
game_over=False
cap = cv2.VideoCapture(0)
score=0
img=np.zeros(shape=(700, 640, 3))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        score=0
        print("Error reading frame")
        break
    if game_over:
        img=np.zeros(shape=(700, 640, 3))
        cv2.putText(img, "Game Over", (int(640/2)-100, int(700/2)), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)
        cv2.imshow('game', img)
        if cv2.waitKey(1) & 0xFF == ord('a'):
            strart=True
            game_over=False
    else:
        
        if start:
            start=False
            img=np.zeros(shape=(1200, 640, 3))
            posx=int(img.shape[1]/2)
            posy=img.shape[0]-270
            voiture.posX=posx
            voiture.posY=posy
            score=0
            img=place_car(img,voiture.posX,voiture.posY,voiture.img)
            img=place_meteore(img,liste_meteor)
            img=place_meteore(img,list_star)
            image_afficher=img[300:-200,:,:]
            scoreborder=np.zeros((700,200,3))
            cv2.putText(scoreborder,"score :"+str(score),(10,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
            cv2.imshow('score', scoreborder)
            cv2.imshow('game', image_afficher)
            
            
            
        else:      
            img=remove_car(img,voiture.posX,voiture.posY,voiture.img)
            img=remove_meteore(img,liste_meteor)
            img=remove_meteore(img,list_star)
            if keyboard.is_pressed('g'):
                voiture.posX-=10
            if keyboard.is_pressed('h'):
                voiture.posX+=10
            
            for i in liste_meteor:
                i.update()
                if i.posX-i.radius-10<0 or i.posX+i.radius+10>img.shape[1]:
                    i.update_moveX(-(i.moveX))
                if i.posY-i.radius>img.shape[0]:
                    i.update_posY(0)
            for i in liste_meteor:
                posvm=int((voiture.posX-i.posX)*(voiture.posX-i.posX)+(voiture.posY-i.posY)*(voiture.posY-i.posY))
                posvm=int(math.sqrt(posvm))
                if(posvm<voiture.radius+i.radius):
                    game_over=True
            for i in list_star:
                i.update()
                if i.posX<0 or i.posX>img.shape[1]:
                    i.update_moveX(-(i.moveX))
                if i.posY>1000:
                    i.update_posY(0)
            for i in list_star:
                posvm=int((voiture.posX-i.posX)*(voiture.posX-i.posX)+(voiture.posY-i.posY)*(voiture.posY-i.posY))
                posvm=int(math.sqrt(posvm))
                if(posvm<voiture.radius+i.radius):
                    score+=1
                    i.update_posY(0)

            img=place_car(img,voiture.posX,voiture.posY,voiture.img)
            img=place_meteore(img,liste_meteor)
            img=place_meteore(img,list_star)
            image_afficher=img[300:-200,:,:]
            scoreborder=np.zeros((700,200,3))
            cv2.putText(scoreborder,"score :"+str(score),(10,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
            img
            cv2.imshow('score', scoreborder)
            cv2.imshow('game', image_afficher)
            
            

    # Read the frame from the webcam
    
    
    


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ValueError: operands could not be broadcast together with shapes (700,640,3) (700,200,3) 

In [191]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
image_afficher.shape

(700, 640, 3)